In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
#plt.style.use('fivethirtyeight')
plt.rc('figure', figsize=(13, 7))
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import sklearn.preprocessing
scaler = sklearn.preprocessing.MinMaxScaler()
from scipy import stats

from pandas.plotting import register_matplotlib_converters
#from statsmodels.tsa.api import Holt

#from acquire import get_store_data, opsd_germany_daily
#import prepare

### Bringing in 5yr Daily historical data of BTC and CNY

In [2]:
cny = pd.read_csv('USD_CNY.csv',sep=',', skipinitialspace=True)
cny.head()

,Date,Price,Open,High,Low,Change %
0,"Dec 04, 2020",6.5315,6.5430,6.5523,6.5271,-0.16%
1,"Dec 03, 2020",6.5418,6.5540,6.5664,6.5423,-0.31%
2,"Dec 02, 2020",6.5622,6.5639,6.5692,6.5519,-0.13%
3,"Dec 01, 2020",6.5705,6.5789,6.5875,6.5606,-0.08%
4,"Nov 30, 2020",6.5760,6.5768,6.5950,6.5708,0.03%


In [3]:
#cny.Date = pd.to_datetime(cny.Date)

cny.drop(columns = ['Change %',
                  'Open',
                  'High',
                  'Low',], inplace = True)
cny = cny.rename(columns={'Price':'y',
                  'Date':'ds'})
#cny = cny[~(cny['date'] < '2015-12-04')]

In [4]:
#cny = cny.set_index('Date')
#cny.y = cny.y.values[::-1]
cny = cny.iloc[::-1]

In [6]:
cny['ds'] = cny.ds.astype('int64')

ValueError: invalid literal for int() with base 10: 'Dec 04, 2015'

In [ ]:
cny['ds'] = 

In [ ]:
btc = pd.read_csv('BTC-USD.csv',sep=',', skipinitialspace=True)
btc.head()

In [ ]:
#btc.Date = pd.to_datetime(btc.Date)
#btc = btc.set_index('Date')

In [ ]:
btc.tail()

In [ ]:
#btc.Date = pd.to_datetime(btc.Date)

btc.drop(columns = ['Adj Close',
                  'Open',
                  'High',
                  'Low',
                   'Volume'], inplace = True)
#btc = btc.rename(columns={'Close':'x',
                  #'Date':'ds'})

#btc = btc.set_index('Date')

In [ ]:
btc.head()

In [ ]:
btc.info()

In [ ]:
# sneaky, sneaky... the price was labeled as an object instead of float
# brought in a new csv... other one had missing values
#btc['y'] = pd.to_numeric(btc.y, errors='coerce')

In [ ]:
btc.head()
btc.info()

In [ ]:
cny.head()
#cny.tail()

In [ ]:
# ffilling the nan's didnt work here but does work after the 2 df's are joined.
#cny = cny.fillna(method='ffill')

### Not much to prep with stock data, time to Explore!

In [ ]:
# scaling the data using a min max scaler
scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(btc)
scaler.fit(cny)


In [ ]:
# combining the btc and cny data together
#btc['value'] = cny['value']
#df = pd.concat([btc, cny], axis=1)
#df=pd.merge(btc,cny, how='inner', left_index=True, right_index=True)

In [ ]:
#df = df.rename(columns={'Close':'btc',
#                  'y':'cny'})

In [ ]:
#df.Price = df.Price.values[::-1]

In [ ]:
# handling some nulls by ffilling them

#df.fillna(method='ffill', inplace = True)

In [ ]:
#df.tail()

In [ ]:
#df.isnull().sum()

In [ ]:
#scaler.fit(df)
#df_scaled = pd.DataFrame(scaler.transform(df), columns = df.columns, index = df.index)

In [ ]:
df_scaled

In [ ]:
df_scaled.plot()

# Hypothesis Testing - Is there a relationship with CNY and BTC?

 $H_o$ - There is no relationship with the Chinese Yuan and Bitcoin

 $H_a$ - There is a relationship with the Chinese Yuan and Bitcoin

 $a = .05$

In [ ]:
r, p = stats.pearsonr(df.btc, df.cny)
print('r = ',r)
print('p = ',p,'\n')

print('Conclusion: We reject the null hypothesis that there is no relationship between the Chinese Yuan and Bitcoin.')

# Lets use the FB Prophet to predict if these currencies will keep going inversely of eachother

### Time to Split (the data)

In [ ]:
##### Fresh CNY Data #####

cny = pd.read_csv('USD_CNY.csv',sep=',', skipinitialspace=True)
cny.head()

In [ ]:
cny.Date = pd.to_datetime(cny.Date)

cny.drop(columns = ['Change %',
                  'Open',
                  'High',
                  'Low',], inplace = True)
cny = cny.rename(columns={'Price':'y',
                  'Date':'ds'})

In [ ]:
m = Prophet()
m.fit(cny)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
##### Fresh BTC data #####

In [ ]:
btc = pd.read_csv('BTC-USD.csv',sep=',', skipinitialspace=True)
btc.head()

In [ ]:
btc.Date = pd.to_datetime(btc.Date)

btc.drop(columns = ['Adj Close',
                  'Open',
                  'High',
                  'Low',
                   'Volume'], inplace = True)

btc = btc.rename(columns={'Close':'y',
                  'Date':'ds'})

btc.head()

In [ ]:
m0 = Prophet()
m0.fit(btc)

future0 = m0.make_future_dataframe(periods=365)
future0.tail()

forecast0 = m0.predict(future)
forecast0[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m0.plot(forecast0)